In [373]:
import pyreadr

train = pyreadr.read_r('data_train.rds') # also works for RData
test = pyreadr.read_r('data_test.rds') # also works for RData



In [374]:
import pandas as pd
import gc
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, accuracy_score, plot_roc_curve
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt

In [375]:
train_df = train[None]

In [376]:
test_df = test[None]

In [377]:
mapping = {'M': 1, 'F': 0}
train_df.replace({'sex': mapping},inplace=True)
test_df.replace({'sex': mapping},inplace=True)

mapping2 = {'AD': 0, 'SC': 1}
train_df.replace({'histology': mapping2},inplace=True)

In [378]:
#train_df.describe()

In [379]:
#one_hot = pd.get_dummies(train_df['sex'])
#one_hot = one_hot.rename(columns={0: 'female',1: 'male'})

In [380]:
labels=train_df['histology'].values

In [381]:
train_cont = train_df.iloc[:,3:].copy()

In [382]:
test_cont = test_df.iloc[:,3:].copy()

In [383]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train_cont)
# Apply transform to both the training set and the test set.
train_cont = scaler.transform(train_cont)
test_cont = scaler.transform(test_cont)

In [384]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(train_cont)
train_cont = pca.transform(train_cont)
test_cont = pca.transform(test_cont)

In [385]:
test_cont.shape

(100, 260)

In [386]:
train_df['age2'] = train_df['age']
test_df['age2'] = test_df['age']
scaler.fit(train_df[['age','age2']])
# Apply transform to both the training set and the test set.
train_oth = scaler.transform(train_df[['age','age2']])
test_oth = scaler.transform(test_df[['age','age2']])

In [387]:
train_oth[:,1]=train_df['sex'].values
test_oth[:,1]=test_df['sex'].values

In [388]:
from scipy import stats
def T_Test(features,labels):
    t_test = [] #prealocate
    for y in range(0,np.shape(features)[1]):
        t_test.append(stats.ttest_ind(features[np.where(labels==1),y][0],features[np.where(labels==0),y][0])[0]) # perform t-test for each feature and record result
    t_test = np.absolute(np.array(t_test)) #take absoolute values of t-test statistic
    feature_set = t_test.argsort()[::-1][:50] # find rank idexes from lowest to highest, reverse an array and take 10 first elements - indecies of 10 highest t statistics
    return feature_set

In [389]:
feature_set = T_Test(features=train_cont,labels=labels)

In [390]:
train_cont = train_cont[:,feature_set]

In [391]:
test_cont = test_cont[:,feature_set]

In [392]:
train_cont.shape

(546, 50)

In [393]:
X_train = np.hstack((train_oth,train_cont))

In [394]:
X_test = np.hstack((test_oth,test_cont))

In [395]:
X_test.shape

(100, 52)

In [396]:
#from sklearn.svm import SVC
#classifier = SVC(kernel="rbf")

# Train it on the entire training data set
#classifier.fit(X_train, labels)

# Get predictions on the test set
#y_pred = classifier.predict(X_test)

In [397]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = SVC(kernel="rbf")
# evaluate model
scores = cross_val_score(model, X_train, labels, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
model.fit(X_train, labels)

# Get predictions on the test set
y_pred = model.predict(X_test)

Accuracy: 0.989 (0.012)


In [399]:
y_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0], dtype=int64)

In [401]:
df = pd.DataFrame(y_pred, 
             columns=['predictions'])

In [404]:
df

,predictions
0,AD
1,AD
2,SC
3,AD
4,AD
...,...
95,SC
96,SC
97,AD
98,SC


In [403]:
mapping3 = {0:'AD', 1:'SC'}
df.replace({'predictions': mapping3},inplace=True)

In [407]:
df.to_csv('results.txt', index=False,header=False)